<a href="https://colab.research.google.com/github/ismailozcil/affordance_git/blob/main/git_webcam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image as pil_im
%matplotlib inline
import  scipy.io

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.models as models
from torchvision import datasets
import torchvision.transforms as transforms
from torchvision.io import read_image

from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
from IPython.display import Image


device = torch.device('cpu')#('cuda:0')

class SquarePad:
    def __call__(self, image):
        s = image.shape
        max_wh = max(s[-1], s[-2])
        hp = int((max_wh - s[-1]) / 2)
        vp = int((max_wh - s[-2]) / 2)
        padding = (hp, hp, vp, vp)
        return F.pad(image, padding, 'constant', 0)

class norm_to_zo:
    def __call__(self, image):
        return image/255.0


def get_net_out(img):
    model_conv = models.regnet_y_16gf(pretrained = True)
    for param in model_conv.parameters():
        param.requires_grad = False

    model_feature = torch.nn.Sequential(*(list(model_conv.children())[:-1]))
    model_feature = model_feature.to(device)
    model_feature.eval()

    transform = transforms.Compose([
    transforms.ToTensor(),
    SquarePad(),
    #norm_to_zo(),
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
    ])
    img = transform(img)
    img = img.unsqueeze(0)
    img = img.to(device)
    with torch.no_grad():
        img_tens = model_feature(img)
    return img_tens

def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
        async function takePhoto(quality) {
            const div = document.createElement('div');
            const capture = document.createElement('button');
            capture.textContent = 'Capture';
            div.appendChild(capture);

            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});

            document.body.appendChild(div);
            div.appendChild(video);
            video.srcObject = stream;
            await video.play();

            // Resize the output to fit the video element.
            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

            // Wait for Capture to be clicked.
            await new Promise((resolve) => capture.onclick = resolve);

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getVideoTracks()[0].stop();
            div.remove();
            return canvas.toDataURL('image/jpeg', quality);
        }
        ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    pil_img = pil_im.open(filename)

# each affordance class is set to live 20 dim subspace
nr_of_bases = 20
img_size = 224
# object names of the dataset are defined
object_labels = ['ball', 'binder', 'bowl', 'calculator', 'camera','cap', 'cell_phone',
                'cereal_box', 'coffee_mug','comb', 'dry_battery', 'flashlight', 'food_bag',
                'food_box', 'food_can', 'food_cup', 'food_jar', 'glue_stick', 'hand_towel',
                'instant_noodles', 'keyboard','kleenex', 'lightbulb', 'marker', 'notebook',
                'pitcher', 'plate', 'pliers', 'rubber_eraser', 'scissors', 'shampoo',
                 'soda_can', 'sponge', 'stapler', 'toothbrush', 'toothpaste', 'water_bottle']

# listing affordance names to make a dictionary that maps affordance label values to names
affordance_names_T = ['Grasp', 'Wrap Grasp', 'Contain', 'Open', 'Tip-push', 'Display',
                    'Roll', 'Dry', 'Liquid contain', 'Pour', 'Grip', 'Absorb',
                    'Cut', 'Staple', 'Illuminate']
affordance_names = ['grasp', 'wrap grasp', 'containment', 'openable', 'tip-push', 'display',
                    'rollable', 'dry', 'liquid_containment', 'pourable', 'grip', 'absorb',
                    'cut', 'stapling', 'illumination']
#temporary threshold values, they will be calculated more precisely later
threshold_val = 0.65
threshold_val2 = 0.9
threshold_val3 = 0.5

#model_name = 'regnet_y'
#model_name_T = 'RegNetY'
model_name = 'resnet18'
model_name_T = 'ResNet18'
dataset_name = 'RGBDAffordance'


def get_tens(net_name, datas_name, im_size):
    # state vectors obtained from Resnet are loaded from previously saved file
    st_tns = torch.load(r'/content/%s_with_%s_%dnetworkout.pt'%(net_name, datas_name, im_size))
    st_tns = torch.transpose(st_tns, 0, 1)
    # also results of those tensor are loaded. Results contain affordance labels, like 0, 1, 2,
    rslt_tns = torch.load(r'/content/%s_with_%s_%dlabel.pt'%(net_name, datas_name, im_size))

    # image paths are loaded to name list
    # image names also contain names of the object/s in the image
    nm_list = []
    f = open(r'/content/%s_with_%s_%dimagepaths.txt'%(net_name, datas_name, im_size), 'r')
    for line in f:
        nm_list= line.strip().split('#')
    f.close()

    return st_tns, rslt_tns, nm_list

def project_matr(bases):
    return torch.matmul(torch.matmul(bases, torch.linalg.inv(
        torch.matmul(torch.transpose(bases, 0, 1), bases) )), torch.transpose(bases, 0, 1))

def matr_to_origin(tens, indx):
    [dim1, dim2] = tens.shape
    t = tens[:, indx]
    t = t.resize(dim1, 1).expand(dim1, dim2)
    return tens-t

def matr_vec_to_origin(tens, vec):
    [dim1, dim2] = tens.shape
    vec = vec.resize(dim1, 1).expand(dim1, dim2)
    return tens-vec

def matr_zero_mean(M):
    [dim1, dim2] = M.shape
    mn = torch.mean(M, 1)
    mn = mn.resize(dim1, 1).expand(dim1, dim2)
    return M-mn

def calc_mi(m, ind,rdim, nnum):
    mto = matr_to_origin(m, ind)
    dig = torch.diag(torch.matmul(torch.transpose(mto, 0, 1), mto))
    salad, ind = torch.sort(dig)
    ind_e = ind[1:nnum+1]
    mi= matr_zero_mean(m[:, ind_e])
    u, s, vh = torch.linalg.svd(mi, full_matrices=True)
    return( torch.matmul(u[:,:rdim], torch.diag(s[:rdim])), s[:rdim], ind_e)

def curv_calc(m, vct, rdim, nnum):
    mtp = matr_vec_to_origin(m, vct)
    dig = torch.diag(torch.matmul(torch.transpose(mtp, 0, 1), mtp))
    salad, ind = torch.sort(dig)
    ind_e = ind[0:nnum]
    ind_w = ind[1:nnum]
    mn = m[:, ind_e]
    mni = matr_zero_mean(mn)
    #mw = m[:, ind_w]
    #mwi = matr_zero_mean(mw)
    mtot = torch.concat((mn, vct), 1)
    mtoti = matr_zero_mean(mtot)

    un, sn, vn = torch.linalg.svd(mni, full_matrices = True)
    utot, stot, vtot = torch.linalg.svd(mtoti, full_matrices = True)
    #uw, sw, vw = torch.linalg.svd(mwi, full_matrices = True)

    usn = torch.matmul(un[:,:rdim], torch.diag(sn[:rdim]))
    ustot = torch.matmul(utot[:,:rdim], torch.diag(stot[:rdim]))
    #usw = torch.matmul(uw[:,:rdim], torch.diag(sw[:rdim]))

    Q = torch.matmul(torch.transpose(usn, 0, 1), ustot)
    uq, sq, vq = torch.linalg.svd(Q)
    theta = torch.acos(torch.abs(torch.clamp(torch.sum(sq)/torch.sum(sn[:rdim]*stot[:rdim]), min=-1.0, max=1.0)))
    return theta#/theta_w

def curv_calc_auto(m, vct, thresh_vl):
    mtp = matr_vec_to_origin(m, vct)
    dig = torch.diag(torch.matmul(torch.transpose(mtp, 0, 1), mtp))
    salad, ind = torch.sort(dig)

    neighbour_number_list = list()
    neighbour_dim_list = list()

    for indx_num in range(2,100):
        #print(indx_num)
        ind_e = ind[0:indx_num]
        ind_w = ind[1:indx_num]
        mn = m[:, ind_e]
        mni = matr_zero_mean(mn)
        #mw = m[:, ind_w]
        #mwi = matr_zero_mean(mw)
        mtot = torch.concat((mn, vct), 1)
        mtoti = matr_zero_mean(mtot)

        un, sn, vn = torch.linalg.svd(mni, full_matrices = True)
        utot, stot, vtot = torch.linalg.svd(mtoti, full_matrices = True)
        #uw, sw, vw = torch.linalg.svd(mwi, full_matrices = True)
        #print(sn)

        energy_tensor = torch.cumsum(sn, dim = 0)/torch.sum(sn, dim = 0)
        #print(energy_tensor)
        #print(energy_tensor)
        try:
            rdim = torch.min((energy_tensor>thresh_vl).nonzero().squeeze()).item()
        except:
            rdim = 0
        neighbour_number_list.append(indx_num)
        neighbour_dim_list.append(rdim)
        if 3<=indx_num:
            if rdim<=neighbour_dim_list[-2]:
                break




    usn = torch.matmul(un[:,:rdim], torch.diag(sn[:rdim]))
    ustot = torch.matmul(utot[:,:rdim], torch.diag(stot[:rdim]))
    #usw = torch.matmul(uw[:,:rdim], torch.diag(sw[:rdim]))
    """

    Qw = torch.matmul(torch.transpose(usn, 0, 1), usw)
    uqw, sqw, vqw = torch.linalg.svd(Qw)
    theta_w = torch.acos(torch.abs(torch.clamp(torch.sum(sqw)/torch.sum(sn[:rdim]*sw[:rdim]), min=-1.0, max=1.0)))
    """

    Q = torch.matmul(torch.transpose(usn, 0, 1), ustot)
    uq, sq, vq = torch.linalg.svd(Q)
    theta = torch.acos(torch.abs(torch.clamp(torch.sum(sq)/torch.sum(sn[:rdim]*stot[:rdim]), min=-1.0, max=1.0)))
    return theta#/theta_w

def optimal_thresh(prjct_matr, origin_zero, non_origin_zero):
    prjcts = torch.matmul(prjct_matr, origin_zero)
    non_prjcts = torch.matmul(prjct_matr, non_origin_zero)
    ratio_vls = torch.div(torch.norm(prjcts, dim = 0), torch.norm(origin_zero, dim = 0))
    non_ratio_vls = torch.div(torch.norm(non_prjcts, dim = 0), torch.norm(non_origin_zero, dim = 0))

    ref_range = 100
    true_pos_rat = []
    false_pos_rat = []
    ref_list = []
    opt_list = []

    for k in range(ref_range):
        ref_val = k/ref_range
        ref_list.append(ref_val)
        true_pos = torch.sum(ratio_vls>ref_val).item()
        false_neg = torch.sum(ratio_vls<=ref_val).item()

        true_neg = torch.sum(non_ratio_vls<=ref_val).item()
        false_pos = torch.sum(non_ratio_vls>ref_val).item()

        tpr = true_pos/(true_pos+false_neg)
        true_pos_rat.append(tpr)
        fpr = false_pos/(false_pos+true_neg)
        false_pos_rat.append(fpr)
        opt_list.append((fpr**2)+(1-tpr)**2)
        #print('tp, fn, tn, fp:',true_pos, false_neg, true_neg, false_pos)
    save_thresh = ref_list[opt_list.index(min(opt_list))]
    return(ratio_vls, non_ratio_vls, ref_list, opt_list, save_thresh, true_pos_rat, false_pos_rat)

def get_subspace_bases():
    state_tens, res_tens, name_list = get_tens(model_name, dataset_name, img_size)
    #taking transpose of the state tensor to make a column matrix
    # all results are looked up and affordance classes are listed
    afford_labellist = res_tens.unique().tolist()
    # '0' is removed since all results include it
    afford_labellist.remove(0)
    # defining affordance label value to name dictionary
    afford_dict = dict()
    afford_dict_T = dict()

    for i in range(len(afford_labellist)):
        afford_dict[afford_labellist[i]] = affordance_names[i]
        afford_dict_T[afford_labellist[i]] = affordance_names_T[i]

    # some empty dictionaries to be used later
    base_list = dict()
    base_point_vecs = dict()
    non_base_point_vecs = dict()
    state_dict = dict()
    non_state_dict = dict()
    threshold_dict = dict()
    ratio_vals = dict()
    non_ratio_vals = dict()
    nr_of_bases_dict = dict()
    subs_angls = dict()


    red_dim = 3
    num_n = 10
    angl_list_tot = []
    plt.figure()
    for i in afford_labellist:
        print(i)
        indices = torch.nonzero(torch.sum( (res_tens == i).int() , axis = 1))
        non_indices = torch.nonzero(torch.sum( (res_tens == i).int() , axis = 1)==0)
        afford_states = state_tens[:, indices.squeeze()]
        non_afford_states = state_tens[:, non_indices.squeeze()]
        state_dict[i] = afford_states.to(device)

        mean_val = torch.mean(afford_states,1)
        non_mean_val = torch.mean(non_afford_states, 1).to(device)
        base_point_vecs[i] = mean_val.to(device)
        [dim1, dim2] = afford_states.shape
        base_tens = mean_val.resize(dim1, 1).expand(dim1, dim2)
        origin_zero_matr = afford_states-base_tens
        U, S, Vh = torch.linalg.svd(origin_zero_matr, full_matrices = True)
        ratio_tens = torch.zeros(afford_states.shape)
        non_ratio_tens = torch.zeros(non_afford_states.shape)
        all_ratio_tens = torch.zeros(torch.cat((ratio_tens, non_ratio_tens), 1).shape)

        [dim1, dim2] = non_afford_states.shape
        non_base_origin = non_mean_val.resize(dim1, 1).expand(dim1, dim2)
        non_origin_tozero_matr = non_afford_states.to(device)-non_base_origin


        ratio_tens = torch.abs(torch.div(torch.matmul(torch.transpose(U,0,1), origin_zero_matr), torch.norm(origin_zero_matr, dim = 0).unsqueeze(0).expand(origin_zero_matr.shape[0],-1)))
        non_ratio_tens = torch.abs(torch.div(torch.matmul(torch.transpose(U,0,1), non_origin_tozero_matr), torch.norm(non_origin_tozero_matr, dim = 0).unsqueeze(0).expand(non_origin_tozero_matr.shape[0],-1)))
        state_proj_mean = torch.mean(ratio_tens, 1)
        non_state_proj_mean = torch.mean(non_ratio_tens, 1)
        indc = torch.nonzero(state_proj_mean>non_state_proj_mean).squeeze().int()
        rtls, non_ratls, ref_list, opt_list, save_thresh, true_pos_rat, false_pos_rat = optimal_thresh(project_matr( U[:,indc]).to(device), origin_zero_matr, non_origin_tozero_matr)
        threshold_dict[i] = save_thresh
        base_list[i] =project_matr( U[:, indc]).to(device)
        plt.plot(false_pos_rat, true_pos_rat, linewidth=3.0, label = afford_dict_T[i])
        plt.scatter(false_pos_rat[opt_list.index(min(opt_list))], true_pos_rat[opt_list.index(min(opt_list))], color = 'r', zorder =1000, s = 60)


    plt.xlabel('False Positive Ratio')
    plt.ylabel('True Positive Ratio')
    plt.grid()
    #plt.legend()
    plt.title('ROC Curves of Affordance Groups for %s'%model_name_T)
    plt.savefig('ROC_Curve_%s.png'%model_name_T.replace("/", ""))
    return state_dict, afford_labellist, afford_dict, afford_dict_T, base_list, base_point_vecs, threshold_dict, ratio_tens, non_ratio_tens